# Cost of skipping a learning day

I fine myself when I skip studying. Started this on [Feb 2, 2024](https://t.me/dkstkn/277) on Telegram.

With 30 subscribers and a goal of 150 mins/day:
- 300₽ for first skip in a month
- 600₽ for second skip
- etc.

Records stored in JSON file.

In [255]:
from datetime import datetime
import json
from pathlib import Path


def calculate_penalty(subscribers, target, days_under_target, min_learned, base_rate=10):
    """Calculate penalty based on learning metrics and subscriber count."""
    return base_rate * subscribers * days_under_target * (1 - min_learned / target)


def add_penalty(
    penalties_dict, amount, subscribers, target, days_under_target, min_learned
):
    """Record a new penalty with today's date."""
    today = datetime.now().strftime("%Y-%m-%d")
    penalties_dict[today] = {
        "amount": amount,
        "subscribers": subscribers,
        "target": target,
        "days_under_target": days_under_target,
        "min_learned": min_learned,
    }
    return today


def get_total_penalties(penalties_dict):
    """Sum all penalty amounts."""
    return sum(entry["amount"] for entry in penalties_dict.values())


def save_penalties(penalties_dict, filename="study_skip_penalties.json"):
    """Save penalties dictionary to a JSON file."""
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(penalties_dict, f, indent=2, ensure_ascii=False)
    print(f"Record saved to {filename}")


def load_penalties(filename="study_skip_penalties.json"):
    """Load penalties dictionary from a JSON file."""
    file_path = Path(filename)
    if file_path.exists():
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    else:
        return {}

In [256]:
# current values. Update these variables to calculate current penalty
current_subs = 30
current_target = 150
days_under_target = 0  # Set to 0 if no penalty, or 1, 2, etc. for penalty coefficient
min_learned = 0  # How many minutes you actually studied

In [257]:
# Load previous penalties
file_path = Path("study_skip_penalties.json")
if file_path.exists():
    unspent_penalties = load_penalties()
    print(f"Previous {len(unspent_penalties)} values loaded.")
else:
    # Initialize with empty dictionary if file doesn't exist
    unspent_penalties = {}
    print("No previous penalties found. Starting with empty record.")

Previous 4 values loaded.


In [258]:
# Calculate and possibly add today's penalty
today = datetime.now().strftime("%Y-%m-%d")
if days_under_target > 0:
    # calculate and record new penalty
    skip_cost = calculate_penalty(
        current_subs, current_target, days_under_target, min_learned
    )
    add_penalty(
        unspent_penalties,
        skip_cost,
        current_subs,
        current_target,
        days_under_target,
        min_learned,
    )
    today_message = f"\n{today}: -{skip_cost:.0f} ₽ ({current_subs} subscribers, target: {current_target})"
    # Save the updated records
    save_penalties(unspent_penalties)

else:

    today_message = f"\nNo penalty. Learned more than {current_target} minutes."

# Now display all records in order
print("\nPrevious records:")
for date, info in sorted(unspent_penalties.items()):
    if date != today:  # Don't show today's record yet
        print(
            f"{date}: -{info['amount']:.0f} ₽ ({info['subscribers']} subscribers, target: {info['target']})"
        )

# Display today's record
print(f"\nToday's record: {today_message}")

# Get total for statistics
total_unspent_penalty = get_total_penalties(unspent_penalties)


Previous records:
2025-02-06: -160 ₽ (16 subscribers, target: 120)
2025-03-02: -160 ₽ (16 subscribers, target: 150)
2025-03-27: -580 ₽ (29 subscribers, target: 150)
2025-04-07: -300 ₽ (30 subscribers, target: 150)

Today's record: 
No penalty. Learned more than 150 minutes.


In [259]:
# Calculate statistics
dates = sorted(unspent_penalties.keys())
if dates:
    first_date = datetime.strptime(dates[0], "%Y-%m-%d").date()
    today = datetime.now().date()
    total_days = (today - first_date).days + 1  # +1 to include the first day

    missed_days = sum(1 for entry in unspent_penalties.values() if entry["amount"] > 0)
    consistency = ((total_days - missed_days) / total_days) * 100

    print("\nLearning statistics:\n")
    print(f"Tracking period: {total_days} days. From {first_date} to {today}")
    print(f"Days missed: {missed_days}")
    print(f"Consistency rate: {consistency:.1f}%")
    print(f"Total unpaid penalties: -{total_unspent_penalty:.0f}₽")


Learning statistics:

Tracking period: 64 days. From 2025-02-06 to 2025-04-10
Days missed: 4
Consistency rate: 93.8%
Total unpaid penalties: -1200₽
